In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, SpatialDropout1D

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from keras.utils.np_utils import to_categorical

In [ ]:
voc_size=50000

In [ ]:
path = "/content/drive/MyDrive/dep-nlp/"

In [ ]:
train_emb = np.load(path+'emb/train/train-stack-emb.npy')
train_label = np.load(path+'emb/train/train-stack-label.npy')

In [ ]:
dev_emb = np.load(path+'emb/dev/dev-stack-emb.npy')
dev_label = np.load(path+'emb/dev/dev-stack-label.npy')

In [ ]:
len(train_emb)

In [ ]:
train_emb.shape

In [ ]:
len(dev_emb)

In [ ]:
train_label = to_categorical(train_label, 3)
dev_label = to_categorical(dev_label, 3)

In [ ]:
# unique, counts = np.unique(data, return_counts=True)
# result = np.column_stack((unique, counts)) 
# print (result.shape)

# Test 1 - basic BiLSTM

### part 1

In [ ]:
embedding_vector_features= 768
model=Sequential()

model.add(Bidirectional(LSTM(768)))
# model.add(Dense(1,activation='softmax'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build((None, 1, 768)) #change the value according to embeddings shape
print(model.summary())


In [ ]:
# print(Counter(train_label))
# print(Counter(dev_label))

In [ ]:
print(dev_label)

In [ ]:
print(np.array(train_emb).shape)

### balance the data

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()

In [ ]:
train_emb1, train_label1 = smt.fit_resample(train_emb, train_label)

In [ ]:
# print('Before:', Counter(train_label))
# print('After:', Counter(train_label))

In [ ]:
train_label.shape

In [ ]:
X_train = train_emb1.reshape([-1,1,768]) # change value according to shape
X_test = dev_emb.reshape([-1,1,768])

In [ ]:
model.fit(np.array(X_train),np.array(train_label1),validation_data=(np.array(X_test),np.array(dev_label)),epochs=30,batch_size=64)


In [ ]:
pred = model.predict(X_test)
print(pred)

In [ ]:
from sklearn.metrics import confusion_matrix
# confusion_matrix(y_test,y_pred)
a=0
y_final=[];
for i in y_pred:
  if(i>=0.5):y_final.append(1)
  else:y_final.append(0)
confusion_matrix(y_test,y_final)

In [ ]:
from sklearn.metrics import accuracy_score
# print(y_final)
accuracy_score(y_final, y_test)

### part 2

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(768)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(20), ))
model.add(Dropout(0.2))
model.add(Dense(20))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [ ]:
model.fit(np.array(X_train),np.array(train_label1),validation_data=(np.array(X_test),np.array(dev_label)),epochs=30,batch_size=64)


### one-hot encoding for labels

In [ ]:
y = [0 for x in range(len(train_label))]

for i in range(len(train_label)):
  if(train_label[i]==0):
    train_label[i]=0
    y[i]=0
  if(train_label[i]==1):
    train_label[i]=1
    y[i]=1
  if(train_label[i]==2):
    train_label[i]=2
    y[i]=2

In [ ]:
y_train = tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)

In [ ]:
# print(train_emb, y)

In [ ]:
z = [0 for x in range(len(dev_label))]

for i in range(len(dev_label)):
  if(dev_label[i]==0):
    dev_label[i]=0
    z[i]=0
  if(dev_label[i]==1):
    dev_label[i]=1
    z[i]=1
  if(dev_label[i]==2):
    dev_label[i]=2
    z[i]=2

y_dev = tf.keras.utils.to_categorical(
    z, num_classes=None, dtype='float32'
)

In [ ]:
#print(y_dev)

# Test 2

In [ ]:
X_train = train_emb.reshape(-1, 768, 1) # change value according to shape
X_test = dev_emb.reshape(-1, 768, 1)

In [ ]:
model1 = Sequential()
model1.add(Bidirectional(LSTM(200, input_shape=(8107,768), return_sequences=True)))
model1.add(Dropout(0.3))
model1.add(Bidirectional(LSTM(20)))
model1.add(Dropout(0.2))
model1.add(Dense(20, activation='relu'))
model1.add(Dropout(0.4))
model1.add(Dense(3, activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model1.fit(np.array(X_train),np.array(train_label),validation_data=(np.array(X_test),np.array(dev_label)), epochs=50, batch_size=100)


# Test 3 - XGBoost

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
from xgboost.sklearn import XGBClassifier
#initial model
xgb1 = XGBClassifier(learning_rate=0.1,
                    n_estimators=1000,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softmax',
                    nthread=4,
                    num_class=9,
                    seed=27)

In [ ]:
xgb1.fit(train_emb, train_label)

In [ ]:
pred = xgb1.predict(dev_emb) 

In [ ]:
print("Accuracy for stacked embeddings (without balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred, dev_label))
print(classification_report(pred, dev_label))

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()

In [ ]:
train_emb_smt, train_label_smt = smt.fit_resample(train_emb, train_label)

In [ ]:
print('Before:', Counter(train_label))
print('After:', Counter(train_label_smt))

In [ ]:
xgb1.fit(train_emb_smt, train_label_smt)

In [ ]:
pred_bal = xgb1.predict(dev_emb)

In [ ]:
print("Accuracy for stacked embeddings (after balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred_bal, dev_label))
print(classification_report(pred_bal, dev_label))

# Random Forest Classifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
path = "/content/gdrive/MyDrive/dep-nlp/"

In [ ]:
train = np.load(path+"emb/train/train-stack-emb.npy")
dev = np.load(path+"emb/dev/dev-stack-emb.npy")

In [ ]:
train_label = np.load(path+"emb/train/train-stack-label.npy")
dev_label = np.load(path+"emb/dev/dev-stack-label.npy")

In [ ]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(train, train_label)

In [ ]:
pred = classifier.predict(dev)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, multilabel_confusion_matrix

print("Accuracy for stacked embeddings (without balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred, dev_label))
print(classification_report(pred, dev_label))

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()

In [ ]:
train_emb_smt, train_label_smt = smt.fit_resample(train, train_label)

In [ ]:
from collections import Counter
print('Before:', Counter(train_label))
print('After:', Counter(train_label_smt))

In [ ]:
classifier1 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier1.fit(train_emb_smt, train_label_smt)

In [ ]:
pred_bal = classifier1.predict(dev)

In [ ]:
print("Accuracy for stacked embeddings (after balancing)")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(pred_bal, dev_label))
print(classification_report(pred_bal, dev_label))

# Test 5- LSTM

In [ ]:
path = "/content/drive/MyDrive/dep-nlp/"

In [ ]:
train_emb = np.load(path+'emb/train/train-stack-emb.npy')
train_label = np.load(path+'emb/train/train-stack-label.npy')

In [ ]:
dev_emb = np.load(path+'emb/dev/dev-stack-emb.npy')
dev_label = np.load(path+'emb/dev/dev-stack-label.npy')

In [ ]:
train_emb.shape

In [ ]:
dev_emb.shape

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()

In [ ]:
train_emb_smt, train_label_smt = smt.fit_resample(train_emb, train_label)

In [ ]:
train_label_smt = to_categorical(train_label_smt, 3)
dev_label = to_categorical(dev_label, 3)

In [ ]:
train_emb_smt.shape

In [ ]:
train_label_smt.shape

In [ ]:
dev_label.shape

In [ ]:
x_train = train_emb_smt.reshape(-1,768,1)
y_train = train_label_smt

In [ ]:
x_train.shape

In [ ]:
x_val = dev_emb.reshape(-1,768,1)
y_val = dev_label

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(100, input_shape=(16575, 768), return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(20)))
model.add(Dropout(0.2))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(x_train, y_train, batch_size=100, epochs = 10, validation_data=(x_val, y_val))

# With embedding layer

In [ ]:
model1 = Sequential()
model1.add(Embedding(25000, 768, input_length=x_train.shape[1]))
model1.add(SpatialDropout1D(0.7))
model1.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
model1.add(Dense(3, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model1.fit(x_train, y_train, epochs=10, batch_size=100)


# XLnet

In [ ]:
!pip install simpletransformers --quiet

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


model = MultiLabelClassificationModel('xlnet', 'xlnet-base-cased', num_labels=3, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})


In [ ]:
train = pd.read_csv(path+'data/train.csv')
dev = pd.read_csv(path+'data/dev.csv')

In [ ]:
train = train.drop('id', axis=1)
dev = dev.drop('id', axis=1)

In [ ]:
frames = [train, dev]
df = pd.concat(frames)

In [ ]:
data = pd.get_dummies(df, columns = ['label'])

In [ ]:
data.head()

In [ ]:
possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)
# df['label'] = df.label.replace(label_dict)
#dev_clean['label'] = dev_clean.label.replace(label_dict)


In [ ]:
train_df, eval_df = train_test_split(data, test_size=0.2)

In [ ]:
train_df.head()

In [ ]:
model.train_model(train_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)